In [2]:
''' 
¤ in colab:

!pip install optuna
!pip install import_ipynb
!pip install torchmetrics

'''

' \n¤ in colab:\n\n!pip install optuna\n!pip install import_ipynb\n!pip install torchmetrics\n\n'

In [3]:
%reload_ext autoreload
%autoreload 2

import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

import import_ipynb

c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
''' 
¤ in colab:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/drive/My Drive/Colab Notebooks/IDS_Project'
'''

" \n¤ in colab:\n\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n%cd '/content/drive/My Drive/Colab Notebooks/IDS_Project'\n"

In [5]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
from neural_net_model_train import train_net_model, test_net_model

In [7]:
from utils_data_prep_py import get_dataloaders, get_input_size, get_number_of_classes

DEVICE:  cuda
shape of X:  (2827876, 78)
number of labels of y:  15
Class labels:  ['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']


 After spliting the data:
training data shape: (2120907, 23)
test data shape: (706969, 23)
training data shape: (2120907,)
test data shape: (706969,)


c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project\utils_data_prep_py.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X_train = torch.tensor(X_train, dtype = torch.float32).clone().detach()


In [8]:
train_loader,test_loader = get_dataloaders()

In [9]:
input_size = get_input_size()
number_of_classes = get_number_of_classes()

In [10]:
def weights_init(m):
    if isinstance(m, nn.Conv1d):
      nn.init.kaiming_uniform_(m.weight.data,nonlinearity='leaky_relu')
      if m.bias is not None:
          nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight.data,nonlinearity='leaky_relu')
        if m.bias is not None:
          nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)  

In [ ]:
""" DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# remove Dropout because is still researched how to appply to Conv , some details here:
# https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html


# Build a model by implementing define-by-run design from Optuna
def define_model(trial, input_features, numClasses):
    
    no_layers = trial.suggest_int("n_layers", 2, 7)
    layers = []
    num_groups = 4 
    
    for i in range(no_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i), 25, 257)
        
        layers.append(nn.Conv1d(input_features, out_features,kernel_size= 1)) 
        #num_groups must divide out_features but if out_features is prime than num_groups=1 ,so 1 is good?
        #layers.append(nn.GroupNorm(num_groups, out_features))
        layers.append(nn.MaxPool1d(kernel_size=1))

        # instead of GroupNorm it is used BatchNorm1d because https://www.researchgate.net/publication/361788277_Understanding_and_Improving_Group_Normalization
        layers.append(nn.BatchNorm1d(out_features))
        layers.append(nn.LeakyReLU())
        drop_procentages = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
        # layers.append(nn.Dropout(drop_procentages))
        
        input_features = out_features
        
    drop_procentages = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
    layers.append(nn.LSTM(
            input_size=1, hidden_size = input_features, num_layers=2, batch_first=True,  dropout=drop_procentages, bidirectional=True
        ))
    layers.append(nn.Linear(input_features * 2, numClasses))
        
    net_cnn_bilstm = nn.Sequential(*layers)
    net_cnn_bilstm.apply(weights_init)
        

    def init_func(self):
        super(NetModel, self).__init__()
       
    no_layers=len(layers)
    def forward_func(self, inputs, no_layers=no_layers): 
        outputs=inputs.unsqueeze(2)
        i = 1
        for index in range(1,no_layers + 1):
            outputs = getattr(self, f"cv"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"maxp"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"bn"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"lrel"+ str(index))(outputs) 
            outputs = getattr(self, f"drop"+ str(index))(outputs) 

        h0 = torch.zeros(self.n_layers * 2, outputs.size(0), self.hidden_size_lstm).to(DEVICE)
        c0 = torch.zeros(self.n_layers * 2, outputs.size(0), self.hidden_size_lstm).to(DEVICE)

        outputs = outputs.squeeze(2)
        
        outputs, hidden = getattr(self, f"bi_lstm")(outputs, (h0, c0)) 
        assert not torch.isnan(outputs).any()

        outputs = outputs[:, -1, :]
        outputs, hidden = getattr(self, f"fc_output")(outputs) 
        assert not torch.isnan(outputs).any()
        print('----------finished-------------')
        return outputs
    
    NetModel = type(
    'NetModel',
    (nn.Module,),
    {
        '__init__': init_func,
        'forward': forward_func,
    })
    cnn_bilstm_model = NetModel()

    # super(CNN_BILSTM, self).__init__()
    linear_output = layers.pop()
    bi_lstm = layers.pop()
    
    i = 0
    index=1
    print('----------len(layers)',len(layers))
    while i<len(layers):
        setattr(cnn_bilstm_model,'cv'+str(index),   layers[i])
        setattr(cnn_bilstm_model,'maxp'+str(index), layers[i+1])
        setattr(cnn_bilstm_model,'bn'+str(index),   layers[i+2])
        setattr(cnn_bilstm_model,'lrel'+str(index), layers[i+3])
        setattr(cnn_bilstm_model,'drop'+str(index), layers[i+4])
        index = index + 1
        print(index)
        i = i + 5
            
    setattr(cnn_bilstm_model,'bi_lstm', bi_lstm)
    setattr(cnn_bilstm_model,'fc_output', linear_output)
    
        

    return cnn_bilstm_model """

In [ ]:
""" DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Build a model by implementing define-by-run design from Optuna
def define_model(trial, input_features, numClasses):
    
    no_layers = trial.suggest_int("n_layers", 2, 7)
    layers = []
    num_groups = 4 
    
    
    def init_func(self):
        super(NetModel, self).__init__()
       
    def forward_func(self, inputs, no_layers=no_layers): 
        outputs=inputs.unsqueeze(2)
        
        for index in range(1,no_layers + 1):
            outputs = getattr(self, f"cv"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"avgp"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"bn"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"lrel"+ str(index))(outputs) 

        lstm_layers=2
        h0 = torch.zeros(lstm_layers * 2, outputs.size(0), self.hidden_size_lstm).to(DEVICE)
        c0 = torch.zeros(lstm_layers * 2, outputs.size(0), self.hidden_size_lstm).to(DEVICE)

        outputs1 = outputs.squeeze(2)
        #outputs = outputs.transpose(1, 2)
        
        
        outputs, hidden = getattr(self, f"bi_lstm")(outputs, (h0, c0)) 
        assert not torch.isnan(outputs).any()

        outputs2 = outputs[:, -1, :]
        outputs = outputs.permute(0, 2, 1)[:, -1, :]

        outputs = getattr(self, f"fc_output")(outputs) 
        assert not torch.isnan(outputs).any()

        
        return outputs
    
    NetModel = type(
    'NetModel',
    (nn.Module,),
    {
        '__init__': init_func,
        'forward': forward_func,
    })
    
    cnn_bilstm_model = NetModel()
    
    for i in range(no_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 25, 257)
        
        setattr(cnn_bilstm_model,'cv'+str(i+1),   nn.Conv1d(input_features, out_features,kernel_size= 1)) 
        # layers.append(nn.MaxPool1d(kernel_size=1))
        setattr(cnn_bilstm_model,'avgp'+str(i+1), nn.AvgPool1d(kernel_size=1))
        # setattr(cnn_bilstm_model,'maxp'+str(i+1), nn.MaxPool1d(kernel_size=1))
        setattr(cnn_bilstm_model,'bn'+str(i+1),   nn.BatchNorm1d(out_features))
        setattr(cnn_bilstm_model,'lrel'+str(i+1),nn.LeakyReLU())
        
        input_features = out_features


    setattr(cnn_bilstm_model,'hidden_size_lstm',input_features)
         
    drop_procentages = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
     
    setattr(cnn_bilstm_model,'bi_lstm', nn.LSTM(
            input_size=1, hidden_size = input_features, num_layers=2, batch_first=True,  dropout=drop_procentages, bidirectional=True
        ))
    setattr(cnn_bilstm_model,'fc_output', nn.Linear(input_features, numClasses))

    cnn_bilstm_model.apply(weights_init)

    return cnn_bilstm_model """

In [26]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# remove Dropout because is still researched how to appply to Conv , some details here:
# https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html


# Build a model by implementing define-by-run design from Optuna
def define_model(trial, input_features, numClasses, use_gru_instead_of_lstm=True):
    
    no_layers = trial.suggest_int("n_layers", 2, 7)
    layers = []
    num_groups = 4 
    
    #to be optimized by optuna?
    rnn_stacked_layers=2
    
    
    def init_func(self):
        super(NetModel, self).__init__()
       
    def forward_func(self, inputs, no_layers=no_layers): 
        # add 1 dimension  for the conv1d
        outputs=inputs.unsqueeze(2)
        
        for index in range(1,no_layers + 1):
            outputs = getattr(self, f"cv"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"avgp"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"bn"+ str(index))(outputs) 
            assert not torch.isnan(outputs).any()

            outputs = getattr(self, f"lrel"+ str(index))(outputs) 

        
        
        if use_gru_instead_of_lstm: 
            h0 = torch.zeros(rnn_stacked_layers * 2, outputs.size(0), self.hidden_size_lstm).to(DEVICE)

            # outputs1 = outputs.squeeze(2)
            #outputs = outputs.transpose(1, 2)
            outputs, hidden = getattr(self, f"bi_gru")(outputs, h0)        
        else:
            h0 = torch.zeros(rnn_stacked_layers * 2, outputs.size(0), self.hidden_size_lstm).to(DEVICE)
            c0 = torch.zeros(rnn_stacked_layers * 2, outputs.size(0), self.hidden_size_lstm).to(DEVICE)

            # outputs1 = outputs.squeeze(2)
            #outputs = outputs.transpose(1, 2)
            outputs, hidden = getattr(self, f"bi_lstm")(outputs, (h0, c0)) 
        assert not torch.isnan(outputs).any()

        # outputs2 = outputs[:, -1, :]
        outputs = outputs.permute(0, 2, 1)[:, -1, :]

        outputs = getattr(self, f"fc_output")(outputs) 
        assert not torch.isnan(outputs).any()

        
        return outputs
    
    NetModel = type(
    'NetModel',
    (nn.Module,),
    {
        '__init__': init_func,
        'forward': forward_func,
    })
    
    cnn_bilstm_model = NetModel()
    
    for i in range(no_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 25, 257)
        
        setattr(cnn_bilstm_model,'cv'+str(i+1),   nn.Conv1d(input_features, out_features,kernel_size= 1)) 
        # layers.append(nn.MaxPool1d(kernel_size=1))
        setattr(cnn_bilstm_model,'avgp'+str(i+1), nn.AvgPool1d(kernel_size=1))
        # setattr(cnn_bilstm_model,'maxp'+str(i+1), nn.MaxPool1d(kernel_size=1))
        setattr(cnn_bilstm_model,'bn'+str(i+1),   nn.BatchNorm1d(out_features))
        setattr(cnn_bilstm_model,'lrel'+str(i+1),nn.LeakyReLU())
        
        input_features = out_features


    setattr(cnn_bilstm_model,'hidden_size_lstm',input_features)
         
    drop_procentages = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
     
    if use_gru_instead_of_lstm: 
        # input_size=1 because the data is 1D
        setattr(cnn_bilstm_model,'bi_gru', nn.GRU(
                input_size=1, hidden_size = input_features, num_layers = rnn_stacked_layers, batch_first=True,  dropout=drop_procentages, bidirectional=True
            ))
    else:
        setattr(cnn_bilstm_model,'bi_lstm', nn.LSTM(
                input_size=1, hidden_size = input_features, num_layers = rnn_stacked_layers, batch_first=True,  dropout=drop_procentages, bidirectional=True
            ))   
    
    setattr(cnn_bilstm_model,'fc_output', nn.Linear(input_features, numClasses))

    cnn_bilstm_model.apply(weights_init)

    return cnn_bilstm_model

##### Hyperparameter Tuning with Accuracy Pruning

In [27]:

MAX_EPOCHS = 2
import pprint
# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

    input_features = input_size
    numClasses = number_of_classes
    #trial controls de parameters of the model
    cnn_bilstm_model = define_model(trial, input_features, numClasses,use_gru_instead_of_lstm=True)
    # cnn_bilstm_model= CNN_bilstm_model()
    print((cnn_bilstm_model.hidden_size_lstm))

    print(repr(cnn_bilstm_model))
    # print(type(cnn_bilstm_model))
    # print(dir(cnn_bilstm_model))

    params = {
              'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
    # Generate the optimizers.
    optimizer = getattr(optim, params['optimizer'])(cnn_bilstm_model.parameters(), lr=params['learning_rate'])

    
    # Get the FashionMNIST dataset.
    # train_loader, val_loader = get_mnist()

    losses, accuracies = train_net_model(cnn_bilstm_model, optimizer, train_loader, MAX_EPOCHS, trial_optimisation=trial,trial_parameter='accuracy')
    accuracy, test_loss = test_net_model(cnn_bilstm_model, test_loader)
    print('test loss',test_loss)

    return accuracy
 
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=30)
study.optimize(objective, n_trials=1)

[I 2022-09-09 21:50:43,393] A new study created in memory with name: no-name-b46f0821-ddb4-4e33-8105-69e9e2e387d7


231
NetModel(
  (cv1): Conv1d(23, 159, kernel_size=(1,), stride=(1,))
  (avgp1): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn1): BatchNorm1d(159, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(159, 37, kernel_size=(1,), stride=(1,))
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn2): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(37, 98, kernel_size=(1,), stride=(1,))
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn3): BatchNorm1d(98, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(98, 220, kernel_size=(1,), stride=(1,))
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn4): BatchNorm1d(220, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel4): Leak

  1%|          | 351/65329 [01:17<3:59:19,  4.53it/s]
[W 2022-09-09 21:52:01,276] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\androgo\AppData\Local\Temp\ipykernel_29524\1170637482.py", line 29, in objective
    losses, accuracies = train_net_model(cnn_bilstm_model, optimizer, train_loader, MAX_EPOCHS, trial_optimisation=trial,trial_parameter='accuracy')
  File "c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project\neural_net_model_train.py", line 49, in train_net_model
    outputs = net_model(batch_x)
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\modules\module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "C:\Users\androgo\AppData\Local\Temp\ipykernel_29524\4086871875.

KeyboardInterrupt: 

In [21]:

MAX_EPOCHS = 2
import pprint
# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

    input_features = input_size
    numClasses = number_of_classes
    #trial controls de parameters of the model
    cnn_bilstm_model = define_model(trial, input_features, numClasses,use_gru_instead_of_lstm=False)
    # cnn_bilstm_model= CNN_bilstm_model()
    print((cnn_bilstm_model.hidden_size_lstm))

    print(repr(cnn_bilstm_model))
    # print(type(cnn_bilstm_model))
    # print(dir(cnn_bilstm_model))

    params = {
              'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
    # Generate the optimizers.
    optimizer = getattr(optim, params['optimizer'])(cnn_bilstm_model.parameters(), lr=params['learning_rate'])

    
    # Get the FashionMNIST dataset.
    # train_loader, val_loader = get_mnist()

    losses, accuracies = train_net_model(cnn_bilstm_model, optimizer, train_loader, MAX_EPOCHS, trial_optimisation=trial,trial_parameter='accuracy')
    accuracy, test_loss = test_net_model(cnn_bilstm_model, test_loader)
    print('test loss',test_loss)

    return accuracy
 
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=30)
study.optimize(objective, n_trials=1)

[I 2022-09-09 21:23:35,275] A new study created in memory with name: no-name-4e33784c-41ed-4057-aecf-0f82f959b3bc
c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.204662891427126 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


147
NetModel(
  (cv1): Conv1d(23, 67, kernel_size=(1,), stride=(1,))
  (avgp1): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn1): BatchNorm1d(67, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(67, 111, kernel_size=(1,), stride=(1,))
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn2): BatchNorm1d(111, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(111, 59, kernel_size=(1,), stride=(1,))
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn3): BatchNorm1d(59, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(59, 50, kernel_size=(1,), stride=(1,))
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel4): LeakyR

  0%|          | 0/65329 [00:01<?, ?it/s]
[W 2022-09-09 21:23:36,714] Trial 0 failed because of the following error: RuntimeError('Expected hidden[0] size (2, 32, 147), got [4, 32, 147]')
Traceback (most recent call last):
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\androgo\AppData\Local\Temp\ipykernel_29524\1957460143.py", line 29, in objective
    losses, accuracies = train_net_model(cnn_bilstm_model, optimizer, train_loader, MAX_EPOCHS, trial_optimisation=trial,trial_parameter='accuracy')
  File "c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project\neural_net_model_train.py", line 49, in train_net_model
    outputs = net_model(batch_x)
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\modules\module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "C:\Users\androgo\AppData

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\androgo\AppData\Local\Temp\ipykernel_29524\1957460143.py", line 38, in <cell line: 38>
    study.optimize(objective, n_trials=1)
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\study.py", line 419, in optimize
    _optimize(
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\_optimize.py", line 66, in _optimize
    _optimize_sequential(
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\_optimize.py", line 160, in _optimize_sequential
    frozen_trial = _run_trial(study, func, catch)
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\_optimize.py", line 234, in _run_trial
    raise func_err
  File "c:\Users\androgo\anaconda3\en

In [2]:
def Rob_init(self, name):
    self.name = name

Robot2 = type("Robot2", 
              (), 
              {"counter":0, 
               "__init__": Rob_init,
               "sayHello": lambda self: "Hi, I am " + self.name})

In [3]:
x = Robot2("Marvin")
print(x.name)
print(x.sayHello())

Marvin
Hi, I am Marvin


In [80]:
from ast import arg
from unicodedata import name


def func1(x):
    return 5+x

def solve_for(self, name:str, args):
    do = f"cv2"
    if hasattr(self, do):
            print('true1')
            if callable(func := getattr(self, do)):
                print('true2')
                res=getattr(self, do)(args)
                print(res)

                # func(args)

def init_func(self, color):
        self._color = color
        setattr(self,'cv'+str(2), func1)

def drive(self):
    print("You are driving the car")

Car = type(
    'Car',
    (object,),
    {
        '__init__': init_func,
        'drive': drive,
        'solve_for':solve_for
    })

my_car = Car('red')
my_car.drive()
my_car.__init__('red')
# exec(f"my_car.cv2()")
# my_car.cv2(6)
my_car.solve_for(name='name:str',args= 52)


You are driving the car
true1
true2
57


In [71]:
f"cv2" + str(2)

'cv22'

In [34]:
# Define functions f_0 to f_9
for i in range(10):
    exec(f"def f_{i}(): print({i})")
# Run functions f_0 to f_9
for i in range(10):
    exec(f"f_{i}()")

0
1
2
3
4
5
6
7
8
9


In [ ]:
def solve_for(self, name: str):
    do = f"get_{name}"
    if hasattr(self, do) and callable(getattr(self, do)):
        func = getattr(self, do)
        func()
        
        
def solve_for(self, name: str, *args, **kwargs):
    do = f"get_{name}"
    if hasattr(self, do) and callable(func := getattr(self, do)):
        func(*args, **kwargs)

In [7]:
'y' + str(2)

'y2'

In [8]:
l = list(range(1, 5))
print(l)    # [1, 2, 3, 4]
l.pop()

[1, 2, 3, 4]


4

In [9]:
exec("""def a(x):
...   return x+1""")
a(2)

3

In [12]:
class Foo:
    def __init__(self, v):
        self.v = v
        
def my_new_method(self):
    print("self.v =", self.v)
    
def my_new_method2(self,x):
    print("self.v2 =", self.v+x)

setattr(Foo, 'print_v', my_new_method)
setattr(Foo, 'print_v2', my_new_method2)


Foo(5).print_v()
Foo(5).print_v2(4)

self.v = 5
self.v2 = 9


In [139]:
for index in range(7):
    print(index)

0
1
2
3
4
5
6


In [122]:
for index in range(1,7):
    print(index)


1
2
3
4
5
6
